In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from imblearn import under_sampling
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

from imblearn.datasets import fetch_datasets
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import RUSBoostClassifier

In [2]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
train_data=pd.read_csv('train.csv')

In [4]:
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')
for c in test_data.columns:
    if(test_data[c].dtype==object and c!='Complaint-ID' and c!='Date-received' and c!='Date-sent-to-company'):
        le=LabelEncoder()
        le.fit(train_data[c].astype(str))
        train_data[c]=le.transform(train_data[c].astype(str))
        test_data[c] = test_data[c].map(lambda s: '<unknown>' if s not in le.classes_ else s)
        le.classes_ = np.append(le.classes_, '<unknown>')
        test_data[c]=le.transform(test_data[c].astype(str))
       

In [5]:
train_data['Date-received']=pd.to_datetime(train_data['Date-received'])

train_data['Year']=train_data['Date-received'].dt.year

train_data['Month']=train_data['Date-received'].dt.month

train_data['day']=train_data['Date-received'].dt.day

In [6]:
test_data['Date-received']=pd.to_datetime(test_data['Date-received'])

test_data['Year']=test_data['Date-received'].dt.year

test_data['Month']=test_data['Date-received'].dt.month

test_data['day']=test_data['Date-received'].dt.day


In [7]:
train_data['Complaint-Status']=train_data['Complaint-Status'].astype('category')
train_data['Complaint-Status']=train_data['Complaint-Status'].cat.codes

In [8]:
target=train_data['Complaint-Status']
t_data,d_data,t_label,d_label=train_test_split(train_data,target,test_size=0.10)

# Classifying the data hirarcilly

In [9]:
train_data=train_data.drop(['Complaint-ID','Date-received','Date-sent-to-company','Consumer-complaint-summary'],1)

In [10]:
train_data['Complaint-Status'].value_counts()

1    34300
3     5018
2     2818
0      809
4      321
Name: Complaint-Status, dtype: int64

In [11]:
tr_data=pd.read_csv('train.csv')

In [12]:
tr_data['Complaint-Status'].value_counts()

Closed with explanation            34300
Closed with non-monetary relief     5018
Closed with monetary relief         2818
Closed                               809
Untimely response                    321
Name: Complaint-Status, dtype: int64

In [26]:
train_data['status']=train_data['Complaint-Status'].apply(lambda x: 1 if x==1 else 0)

In [27]:
train_data['status'].value_counts()

1    34300
0     8966
Name: status, dtype: int64

In [28]:
train_data2=train_data[train_data['Complaint-Status']!=1]

In [29]:
required_columns=[u'Transaction-Type', u'Complaint-reason', u'Company-response',
         u'Consumer-disputes', u'Year', u'Month', u'day']

In [30]:
required_columns=[u'Transaction-Type', u'Complaint-reason', u'Company-response',
         u'Consumer-disputes', u'Year', u'Month', u'day']
label=train_data['status']

In [31]:
base_estimator = AdaBoostClassifier(n_estimators=300)

In [32]:
model=xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05,scale_pos_weight=0.8,min_child_weight=1)
model.fit(train_data[required_columns],label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=0.8, seed=None,
       silent=True, subsample=1)

In [33]:
train_data['status'].value_counts()

1    34300
0     8966
Name: status, dtype: int64

In [34]:
cols=train_data2.columns

In [35]:
train_data2=train_data2.drop(['status'],1)
train_data2['status']=train_data2['Complaint-Status'].apply(lambda x:1 if x==3 else 0)

In [36]:
train_data2['status'].value_counts()

1    5018
0    3948
Name: status, dtype: int64

In [38]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
#rus=TomekLinks(return_indices=True, ratio='majority')
rus = RandomUnderSampler(return_indices=True)
label=train_data2['status']
train_data2,label, id_rus = rus.fit_sample(train_data2,label)
train_data2=pd.DataFrame(train_data2)
train_data2.columns=cols
model2=xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05,scale_pos_weight=0.8,min_child_weight=1)
model2.fit(train_data2[required_columns],label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=0.8, seed=None,
       silent=True, subsample=1)

In [39]:
train_data2.shape

(7896, 9)

In [40]:
train_data3=train_data2[train_data2['status']!=1]
train_data3=train_data3.drop(['status'],1)
train_data3['status']=train_data3['Complaint-Status'].apply(lambda x:1 if x==2 else 0)

In [41]:
train_data3['status'].value_counts()

1    2818
0    1130
Name: status, dtype: int64

In [42]:
col_3=['Complaint-reason','Company-response']

In [43]:
cols=train_data3.columns

In [45]:
label=train_data3['status']
from imblearn.under_sampling import RandomUnderSampler
rus=TomekLinks(return_indices=True, ratio='majority')
train_data3,label, id_rus = rus.fit_sample(train_data3,label)
train_data3=pd.DataFrame(train_data3)
train_data3.columns=cols
model3=xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05,scale_pos_weight=0.8,min_child_weight=1)
model3.fit(train_data3[required_columns],label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=0.8, seed=None,
       silent=True, subsample=1)

In [46]:
train_data4=train_data3[train_data3['status']!=1]
train_data4=train_data4.drop(['status'],1)
train_data4['status']=train_data4['Complaint-Status'].apply(lambda x: 1 if x==0 else 0)

In [47]:
train_data4['status'].value_counts()

1    809
0    321
Name: status, dtype: int64

In [48]:
label=train_data4['status']
train_data4,label, id_rus = rus.fit_sample(train_data4,label)
train_data4=pd.DataFrame(train_data4)
train_data4.columns=cols
model4=xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05,scale_pos_weight=0.8,min_child_weight=1)
model4.fit(train_data4[required_columns],label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=0.8, seed=None,
       silent=True, subsample=1)

# Testing 

In [49]:
test_data.shape

(18543, 11)

In [50]:
test_data.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary,Year,Month,day
0,Te-1,2016-08-18,0,1,9,8/18/2016,0,43022,2016,8,18
1,Te-2,2016-04-18,7,33,4,4/20/2016,0,43022,2016,4,18
2,Te-3,2016-03-23,5,71,11,3/23/2016,0,43022,2016,3,23
3,Te-4,2017-06-26,15,49,11,6/26/2017,3,43022,2017,6,26
4,Te-5,2016-05-13,5,71,9,5/13/2016,0,43022,2016,5,13


In [51]:
required_columns.append('Complaint-ID')
#test_data=test_data[required_columns]

In [52]:
test_data=test_data[required_columns]

In [53]:
sub=pd.DataFrame()

In [54]:
tes=pd.read_csv('test.csv')

In [55]:
test_data.head()

,Transaction-Type,Complaint-reason,Company-response,Consumer-disputes,Year,Month,day,Complaint-ID
0,0,1,9,0,2016,8,18,Te-1
1,7,33,4,0,2016,4,18,Te-2
2,5,71,11,0,2016,3,23,Te-3
3,15,49,11,3,2017,6,26,Te-4
4,5,71,9,0,2016,5,13,Te-5


In [56]:
sub['Complaint-ID']=tes['Complaint-ID']

In [57]:
required_columns2=required_columns[:-1]

In [58]:
test_data['status']=model.predict(test_data[required_columns2])

In [59]:
test_data_ans=test_data[test_data['status']==1]

In [60]:
test_data_ans.shape

(17816, 9)

In [61]:
test_data2=test_data[test_data['status']==0]

In [62]:
test_data2=test_data2.drop(['status'],1)

In [63]:
test_data2['status']=model2.predict(test_data2[required_columns2])

In [64]:
test_data2['status'].value_counts()

0    602
1    125
Name: status, dtype: int64

In [65]:
test_data2['status'].value_counts()

0    602
1    125
Name: status, dtype: int64

In [66]:
train_data2['status'].value_counts()

1    3948
0    3948
Name: status, dtype: int64

In [67]:
test_data2['status']=test_data2['status'].apply(lambda x: 3 if x==1 else 0)

In [68]:
test_data_ans2=test_data2[test_data2['status']==3]

In [69]:
test_data3=test_data2[test_data2['status']==0]

In [70]:
test_data3=test_data3.drop(['status'],1)
test_data3['status']=model3.predict(test_data3[required_columns2])

In [71]:
test_data3['status']=test_data3['status'].apply(lambda x: 2 if x==1 else 0)

In [72]:
test_data3['status'].value_counts()

2    601
0      1
Name: status, dtype: int64

In [73]:
test_data3_ans=test_data3[test_data3['status']==2]

In [74]:
test_data3_ans.shape

(601, 9)

In [75]:
test_data=pd.concat([test_data_ans,test_data_ans2,test_data3],0)

In [76]:
test_data.shape

(18543, 9)

In [83]:
test_sub=test_data[['Complaint-ID','status']]

In [84]:
le=LabelEncoder()
train_data_=pd.read_csv('train.csv')
le.fit(train_data_['Complaint-Status'])
test_sub['status']=le.inverse_transform(test_sub['status'])

/home/adarsh/anaconda2/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [85]:
test_sub['status'].value_counts()

Closed with explanation            17816
Closed with monetary relief          601
Closed with non-monetary relief      125
Closed                                 1
Name: status, dtype: int64

In [86]:
sub=pd.merge(sub,test_sub,how='inner',on='Complaint-ID')

In [87]:
sub.columns=['Complaint-ID','Complaint-Status']
sub['Complaint-Status'].value_counts()

Closed with explanation            17816
Closed with monetary relief          601
Closed with non-monetary relief      125
Closed                                 1
Name: Complaint-Status, dtype: int64

In [88]:
sub.to_csv('subbesty1.csv', sep=',',index=False)

In [89]:
sub.head(14)

,Complaint-ID,Complaint-Status
0,Te-1,Closed with explanation
1,Te-2,Closed with explanation
2,Te-3,Closed with explanation
3,Te-4,Closed with explanation
4,Te-5,Closed with explanation
5,Te-6,Closed with monetary relief
6,Te-7,Closed with explanation
7,Te-8,Closed with explanation
8,Te-9,Closed with explanation
9,Te-10,Closed with explanation
